In [1]:
from dataclasses import dataclass, field
import asyncio
import nest_asyncio

nest_asyncio.apply()

event_shutdown_event = asyncio.Event()

@dataclass
class Prams1:
    event_shutdown: asyncio.Event = field(default_factory=lambda: event_shutdown_event)
    event_test1: asyncio.Event = field(default_factory=asyncio.Event)

@dataclass
class Prams2:
    event_shutdown: asyncio.Event = field(default_factory=lambda: event_shutdown_event)
    event_test2: asyncio.Event = field(default_factory=asyncio.Event)

class Test:
    def __init__(self, event_shutdown: asyncio.Event, event_test2: asyncio.Event):
        self.event_shutdown = event_shutdown
        self.event_test2 = event_test2

In [ ]:
import time

event_shutdown_event.clear()

print(Prams1.event_shutdown.is_set())
print(Prams2.event_shutdown.is_set())
time.sleep(5)
event_shutdown.set()
print(Prams1.event_shutdown.is_set())
print(Prams2.event_shutdown.is_set())

AttributeError: type object 'Prams1' has no attribute 'event_shutdown'

In [ ]:
data = a(**args)

In [ ]:
b(data)

In [ ]:
data.a